# Prescriptive Models and Data Analytics Problem Set #2


## Hospital admission & quality of service

**Download health data.csv and load it into python. These are data from hospital admissions for coronary artery bypass graft (CABG) in the UK. Among other things, you observe whether the patient died after the surgery (coded up as patient died dummy), which hospital the patient visited (hospital id), and a series of patient characteristics such as gender and age.**

In [1]:
import pandas as pd
import statsmodels.formula.api as smf

In [2]:
health = pd.read_csv('health_data.csv')

health.head()

,patient_id,hospital_id,admin_year,patient_died_dummy,startage,female_dummy
0,1,D,2003,0,81,0
1,2,H,2003,1,67,0
2,3,A,2003,0,54,0
3,4,E,2003,0,81,0
4,5,H,2003,0,69,0


### Question 1. Start by regressing the patient-died dummy variable on a set of hospital dummies

In [3]:
reg_hospital  = smf.ols(formula = 'patient_died_dummy ~ C(hospital_id)', data = health)
result = reg_hospital.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:     patient_died_dummy   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     119.3
Date:                Sat, 10 Feb 2024   Prob (F-statistic):          1.75e-220
Time:                        23:48:59   Log-Likelihood:                -7416.5
No. Observations:               24480   AIC:                         1.485e+04
Df Residuals:                   24470   BIC:                         1.493e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.0970    

**(a) Based on the regression output, interpret the coefficients on the constant term and the dummy for hospital D**

Constant term (0.097): the average predicted mortality rate when the dummy variables for hospitals B through J are set to zero, which indicates that the mortality rate for hospital A is 9.7%.

Dummy for Hospital D (0.1882): the difference in the average predicted mortality rate between Hopsital D and A. In this case, it indicates that Hospital D has a 18.82% higher mortality rate than Hospital A.

**(b) What is the difference between the mortality rates at hospitals D and E (use the regression output to derive this)?**

The difference between the mortality rates at Hospital D and E is 0.2413 (=coef(D) - coef(E)).

### Question 2. Continue to use the hospital data in this question, but only use data for patients that visited either hospital A or B. Regress mortality on an intercept and a dummy for whether the patient visited hospital B.

In [4]:
health_update = health.loc[health['hospital_id'].isin(['A','B'])] 

health_update.head()

,patient_id,hospital_id,admin_year,patient_died_dummy,startage,female_dummy
2,3,A,2003,0,54,0
6,7,A,2003,0,59,0
12,13,B,2003,0,65,0
15,16,A,2003,0,61,0
19,20,A,2003,0,71,0


In [5]:
reg_hospital_two  = smf.ols(formula = 'patient_died_dummy ~ C(hospital_id)', data = health_update)
result = reg_hospital_two.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:     patient_died_dummy   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9377
Date:                Sat, 10 Feb 2024   Prob (F-statistic):              0.333
Time:                        23:48:59   Log-Likelihood:                -1446.8
No. Observations:                6611   AIC:                             2898.
Df Residuals:                    6609   BIC:                             2911.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.0970    

**(a) Explain why the difference in mortality rate implied by this regression cannot be interpreted as the causal effect of visiting a different hospital (i.e., the change in risk of dying when moving a patient from hospital A to B cannot be inferred from this regression).**

First of all, patients who visit Hospital A or B are not randomly assigned. In other words, Hospital B may only take urgent cases, while Hospital A accepts general patients.

Secondly, there are other factors that may affect the risk of dying besides the choice of Hospital A or B, such as the level of sickness and age.

**(b) Do you think difference in mortality between hospitals are over- or under-estimated? Think about what type of patients go to which type of hospital.**

The difference in mortality between Hospital A and Hospital B could be either underestimated or overestimated due to omitted variable bias:

1. Overestimation: If Hospital B takes more severe and urgent cases but the model does not account for the severity of the illness, then the mortality rate for Hospital B would be higher when compared to Hospital A, thus overestimating the difference between the two hospitals. (Vice Versa)

2. Underestimation: If Hospital A has a superior care but the model does not account for quality of care, which can reduce the mortality rate, thus underestimating the difference between the two hospitals. (Vice Versa)

**(c) What are potential control variables that you might want to include in the regression, in order to obtain a causal estimate (or at least get closer to a causal estimate)? Run such a regression with suitable controls and interpret the change in the coefficient on the hospital B dummy. Explain why you included the specific set of variables.**

I want include age(startage) and gender(female_dummy) in the regression to obtain (or at least get closer to) a causal estimate, as older individuals or females may have a higher mortality rate. By incorporating these two control variables, we can enhance our understanding of the true effect of the hospital dummy variable on the mortality rate through the regression model.

In [6]:
reg_hospital_two_update  = smf.ols(formula = 'patient_died_dummy ~ C(hospital_id)+startage+female_dummy', data = health_update)
result = reg_hospital_two_update.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:     patient_died_dummy   R-squared:                       0.063
Model:                            OLS   Adj. R-squared:                  0.062
Method:                 Least Squares   F-statistic:                     147.6
Date:                Sat, 10 Feb 2024   Prob (F-statistic):           1.43e-92
Time:                        23:48:59   Log-Likelihood:                -1232.8
No. Observations:                6611   AIC:                             2474.
Df Residuals:                    6607   BIC:                             2501.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.1165    

From the regression results above, the coefficient for Hospital B is 0.0114, which is higher than the initial coefficient of 0.0072. This increase in the coefficient after including age and gender suggests that these control variables are significant factors in the model. Possible explanations for this change include: age and gender acting as confounding variables, potential interaction effects between Hospital B and age or gender, and the presence of omitted variable bias, as noted previously.

## Demand estimation

**The dataset demand data.csv contains data on sales and prices at a set of ice-cream vendors measured over 52 weeks. All ice-cream at a given store is always priced the same, so there is only one price variable. However, different vendors charge different prices and most vendors vary their prices throughout the year.**

**Question 1. Load demand data.csv into Python. For vendor 1, run a regression of sales on price and also a regression of sales on price and a summer dummy (make sure your regression selects only the 52 weeks of data for vendor 1). Use the omitted variable bias formula to explain why the price coefficient changes when the summer dummy is also included in the regression.**

In [7]:
demand = pd.read_csv('demand_data.csv')

demand.head()

,vendor_id,week,summer_dummy,price,sales
0,1,1,0,2.0,8788.7383
1,1,2,0,3.0,8937.9863
2,1,3,0,3.0,8740.1777
3,1,4,0,3.0,8757.1338
4,1,5,0,3.0,8739.6104


In [8]:
demand_ven1 = demand[demand['vendor_id'] == 1]

demand_ven1.tail()

,vendor_id,week,summer_dummy,price,sales
47,1,48,0,3.0,8653.0254
48,1,49,0,1.5,8823.1025
49,1,50,0,2.0,8900.7051
50,1,51,0,3.0,8743.8818
51,1,52,0,2.5,8478.7373


In [9]:
reg_ven1  = smf.ols(formula = 'sales ~ price', data = demand_ven1)
result = reg_ven1.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                 -0.013
Method:                 Least Squares   F-statistic:                    0.3250
Date:                Sat, 10 Feb 2024   Prob (F-statistic):              0.571
Time:                        23:48:59   Log-Likelihood:                -360.33
No. Observations:                  52   AIC:                             724.7
Df Residuals:                      50   BIC:                             728.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   8983.8227    145.437     61.771      0.0

In [10]:
reg_ven1_summer  = smf.ols(formula = 'sales ~ price+summer_dummy', data = demand_ven1)
result = reg_ven1_summer.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.318
Model:                            OLS   Adj. R-squared:                  0.290
Method:                 Least Squares   F-statistic:                     11.42
Date:                Sat, 10 Feb 2024   Prob (F-statistic):           8.49e-05
Time:                        23:48:59   Log-Likelihood:                -350.56
No. Observations:                  52   AIC:                             707.1
Df Residuals:                      49   BIC:                             713.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     9177.5500    128.432     71.458   

In [11]:
demand_ven1.corr()

,vendor_id,week,summer_dummy,price,sales
vendor_id,NaN,NaN,NaN,NaN,NaN
week,NaN,1.000000,0.057703,0.032114,0.053877
summer_dummy,NaN,0.057703,1.000000,0.452191,0.461427
price,NaN,0.032114,0.452191,1.000000,-0.080363
sales,NaN,0.053877,0.461427,-0.080363,1.000000


According to the formula for omitted variable bias, the variable 'summer_dummy' is positively correlated with 'price' (0.452191) and is also positively correlated with 'sales' (0.461427). This indicates that there is a positive bias, suggesting that the original regression overstates the effect of 'price' on 'sales'. Consequently, the coefficient of 'price' decreases when 'summer_dummy' is included as a control variable in the regression model.

**Question 2. Repeat the two regressions that you just ran in question 1, but now use data only for vendor 2. In the case of the regression with the summer dummy, you should find that there might be multicollinearity problems. Why does this happen?**

In [12]:
demand_ven2 = demand[demand['vendor_id'] == 2]

demand_ven2.tail()

,vendor_id,week,summer_dummy,price,sales
99,2,48,0,2.5,8845.7813
100,2,49,0,2.5,9492.9307
101,2,50,0,2.5,9357.5439
102,2,51,0,2.5,8901.5459
103,2,52,0,2.5,9063.8477


In [13]:
reg_ven2  = smf.ols(formula = 'sales ~ price', data = demand_ven2)
result = reg_ven2.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.133
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     7.684
Date:                Sat, 10 Feb 2024   Prob (F-statistic):            0.00781
Time:                        23:48:59   Log-Likelihood:                -359.10
No. Observations:                  52   AIC:                             722.2
Df Residuals:                      50   BIC:                             726.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   8411.1748    219.545     38.312      0.0

In [14]:
reg_ven2_summer  = smf.ols(formula = 'sales ~ price+summer_dummy', data = demand_ven2)
result = reg_ven2_summer.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.133
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     7.684
Date:                Sat, 10 Feb 2024   Prob (F-statistic):            0.00781
Time:                        23:48:59   Log-Likelihood:                -359.10
No. Observations:                  52   AIC:                             722.2
Df Residuals:                      50   BIC:                             726.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     2105.3159     29.848     70.534   

In [15]:
demand_ven2.corr()

,vendor_id,week,summer_dummy,price,sales
vendor_id,NaN,NaN,NaN,NaN,NaN
week,NaN,1.000000,0.057703,0.057703,0.152865
summer_dummy,NaN,0.057703,1.000000,1.000000,0.364969
price,NaN,0.057703,1.000000,1.000000,0.364969
sales,NaN,0.152865,0.364969,0.364969,1.000000


In [16]:
df = demand_ven2.groupby('price')['summer_dummy'].mean()

df

price
2.5    0.0
3.5    1.0
Name: summer_dummy, dtype: float64

According to the correlation data, 'price' and 'summer_dummy' are perfectly correlated. This is because the prices from vendor 2 change based on the season, specifically whether or not it is summer. As a result, 'price' can be exactly predicted by 'summer_dummy', leading to multicollinearity in the model.

**Question 3. Suppose that one of the vendors did not systematically charge higher or lower prices in summer. If you were to repeat the analysis you just did for vendors 1 and 2, what would you expect to happen to the price coefficient estimate and its precision in the two regressions with and without the summer dummy?**

If a vendor did not systematically charge higher or lower prices in the summer, indicated by a correlation of zero between 'price' and 'summer_dummy' (corr(price, summer_dummy) = 0) and no omitted variable bias, then I would expect the estimate of the price coefficient to remain similar or unchanged by including 'summer_dummy' in the regression. However, the precision of the estimate would increase.